# 6 задача
## Численно и аналитически решить смешанную задачу для квазилинейного уравнения теплопроводности с точностью 1e-4 и сравнить их значения в 11 точках при t  = 1

### Постановка задачи (вариант №1 задание №2)

#### Дифференциальная задача:

\begin{gather*}
   {\partial u \over \partial t} = {\partial \over \partial x}(u^2 {\partial u \over \partial x})\\
\end{gather*}

\begin{equation}
0 \leq x \leq 1 
\end{equation}

#### Краевые условия задачи:

\begin{equation}
\left.
  \begin{array}{ccc}
      u(x,0) = (1+x) / \sqrt{5}, \;\;\; 0 \leq x \leq 1\\
      u(0,t) = 1 / \sqrt{5-4t}, \;\;\; 0 < t \leq 1\\
      u(0,t) = 2 / \sqrt{5-4t}, \;\;\; 0 < t \leq 1\\
  \end{array}
\right\}
\end{equation}

#### Аналитическое решение:

\begin{equation}
u(x,t) = {1+x \over (5-4t)^{3/2}}
\end{equation}

#### Разностная схема:

\begin{equation}
{u_l^{n+1}-u_l^n \over \tau} = {[(u_{l+1}^n)^2+(u_l^n)^2](u_{l+1}^{n+1}-u_l^{n+1}) \over 2h^2} - {[(u_l^n)^2+(u_{l-1}^n)^2](u_l^{n+1}-u_{l-1}^{n+1}) \over 2h^2}
\end{equation}

Перепишу систему в более удобном виде:
\begin{equation}
-{\tau \over 2h^2}[(u_l^n)^2+(u_{l-1}^n)^2]u_{l-1}^{n+1}+\\
+(1+{\tau \over 2h^2}[u_{l-1}^n)^2-(u_{l+1}^n)^2])u_l^{n+1}-\\
-{\tau \over 2h^2}[(u_l^n)^2+(u_{l+1}^n)^2]u_{l+1}^{n+1} = \\
=u_l^n
\end{equation}

Обозначив коэффициенты в этом уравнении как a,b,c,d, получаю трехдиагональную СЛАУ этой задачи:

\begin{equation} 
a_l^n u_{l-1}^{n+1} + b_l^n u_l^{n+1} + c_l^n u_{l+1}^{n+1} = d_l^n, \; \; \; l \in [0, L]
\end{equation}

In [1]:
import numpy as np
import matplotlib.pyplot
import pandas as pd
import math as m
import scipy as sp

In [2]:
# создать следующий слой по времени:
def nextlay(prevlay, time):
    upper_arr = [-tau/2/h**2*(prevlay[i]**2+prevlay[i+1]**2) for i in range(2,N-2)]
    middle_arr= [1+tau/2/h**2*(prevlay[i-1]**2+prevlay[i+1]**2) for i in range(2,N-2)]
    lower_arr = [-tau/2/h**2*(prevlay[i-1]**2+prevlay[i]**2) for i in range(2,N-2)]

    A = dia_matrix((np.array([lower_arr, middle_arr, upper_arr]), [-1, 0, 1]), shape=(L, L)).toarray()
    B = np.array([d0] + [d(l*h) for l in range(1, L-1)] + [dL])
    solution = np.linalg.solve(A, B) 
    return solution

In [3]:
N = 11
x0, x1 = 0, 1
Nt = 11
Tend = 1
tau = Tend / (Nt-1)
h = (x1-x0) / (N-1)

start_conditions = np.array([(1+i*(x1-x0)/(N-1))/m.sqrt(5) for i in range(0,N)])
boundary_condition0 = np.array([1/m.sqrt(5-4*i*Tend/(Nt-1)) for i in range(0,Nt)])
boundary_condition1 = np.array([2/m.sqrt(5-4*i*Tend/(Nt-1)) for i in range(0,Nt)])

step = 1

In [4]:
current_solution = np.array([(1+i*(x1-x0)/(N-1))/m.sqrt(5) for i in range(0,N)])

In [11]:
A = [1 + (current_solution[i-1]**2 + current_solution[i+1]**2)*tau/2/h**2 for i in range(1,N-1)]
B = [(current_solution[i]**2 + current_solution[i+1]**2)*tau/2/h**2 for i in range(0,N-1)]
C = [(current_solution[i-1]**2 + current_solution[i]**2)*tau/2/h**2 for i in range(1,N)]

triples = np.array([[A[i],B[i+1],C[i]] for i in range(0,N-2)])

diagonals = [A,B[:-1],C[1:]]

MATRIX = sp.sparse.diags(diagonals,[0,1,-1]).toarray()

FREE = create0b(step) + current_solution[2:-2].tolist() + create1b(step)

new_solution = np.linalg.solve(MATRIX, FREE)
    
# step = step+1

In [10]:
def create0b(step):
    return [current_solution[1] - boundary_condition0[step] * C[0]]

def create1b(step):
    return [current_solution[-2] - boundary_condition1[step] * B[-1]]

In [32]:
exactsol = np.array([1+i/10 for i in range(0,11)])

In [14]:
sol64 = new_solution[::64]

In [20]:
sol128 = new_solution[::128]

In [28]:
sol256 = new_solution[::256]

In [40]:
sol512 = new_solution[::512]

In [27]:
pd.DataFrame([sol641,sol1281,sol2561,sol5121])

,0,1,2,3,4,5,6,7,8,9,10
0,1.000463,1.100245,1.200764,1.300886,1.400632,1.500973,1.600017,1.701074,1.801164,1.900366,2.000279
1,1.000097,1.100085,1.200081,1.300087,1.400184,1.500216,1.600114,1.700001,1.800118,1.900167,2.000060
2,1.000039,1.100020,1.200015,1.300027,1.400035,1.500049,1.600042,1.700020,1.800024,1.900024,2.000011
3,1.000000,1.100004,1.200002,1.300010,1.400003,1.500005,1.600003,1.700009,1.800006,1.900007,2.000001


In [34]:
pd.DataFrame([(sol641-exactsol)/exactsol,(sol1281-exactsol)/exactsol,(sol2561-exactsol)/exactsol,(sol5121-exactsol)/exactsol])

,0,1,2,3,4,5,6,7,8,9,10
0,4.634267e-04,0.000223,0.000636,0.000682,0.000451,0.000649,0.000011,6.318912e-04,0.000647,0.000192,1.396033e-04
1,9.711871e-05,0.000077,0.000067,0.000067,0.000132,0.000144,0.000071,3.340510e-07,0.000066,0.000088,2.986804e-05
2,3.850660e-05,0.000018,0.000013,0.000021,0.000025,0.000033,0.000026,1.156635e-05,0.000013,0.000013,5.270600e-06
3,1.261611e-07,0.000004,0.000002,0.000008,0.000002,0.000003,0.000002,5.092290e-06,0.000003,0.000004,4.259578e-07
